In [8]:
import requests
import pandas as pd
from selenium import webdriver
from scrapy.http import TextResponse
import seaborn as sns
import getpass
import time

In [11]:
# 로그인 함수 (seleniun webdriver)

def login():
    login_url = "https://www.instagram.com/accounts/login/?source=auth_switcher"

    driver = webdriver.Chrome()
    driver.get(login_url)
    
    my_id = input('id를 입력하세요: ')
    my_password = getpass.getpass('password를 입력하세요: ')

    driver.find_element_by_css_selector('._2hvTZ.pexuQ.zyHYP[type=text]').send_keys(my_id)
    driver.find_element_by_css_selector('._2hvTZ.pexuQ.zyHYP[type=password]').send_keys(my_password)
    driver.find_element_by_css_selector('.sqdOP.L3NKy.y3zKF[type=submit]').click()
    
    time.sleep(3)
    
    # 알림설정 하라는 modal창이 뜨는경우 '나중에 하기'를 클릭하는 코드
    alert_modal = driver.find_element_by_css_selector('body > div.RnEpo.Yx5HN > div > div')
    if alert_modal:
        driver.find_element_by_css_selector('body > div.RnEpo.Yx5HN > div > div > div.mt3GC > button.aOOlW.HoLwm').click()
    

In [12]:
# 검색어를 입력하고 게시물이 가장 많은 검색결과를 클릭 (seleniun webdriver)

def crawling_keyword(word):
    keyword = word

    driver.find_element_by_css_selector('#react-root > section > nav > div._8MQSO.Cx7Bp > div > div > div.LWmhU._0aCwM > input').send_keys(keyword)

    # 키워드 입력하고, 키워드에 대한 검색 리스트가 뜨는데 걸리는 시간만큼 기다려준다
    time.sleep(1.5)
    
    search_list = driver.find_elements_by_css_selector('#react-root > section > nav > div._8MQSO.Cx7Bp > div > div > div.LWmhU._0aCwM > div:nth-child(4) > div.drKGC > div > a')
    
    # 서칭된 a 엘리먼트들 중에 게시물 갯수가 가장 많은 a 엘리먼트를 뽑는다
    number_list = []

    for element in search_list:
        splited_element = element.text.split('게시물')
        if len(splited_element) == 2:
            number_list.append(int(re.sub(",", "", splited_element[1])))
        else:
            number_list.append(0)

    max_number = max(number_list)
    click_index = number_list.index(max_number)
    
    # 클릭하고 렌더링이 완료되도록 조금 기다리기
    search_list[click_index].click()
    time.sleep(1.5)


In [13]:
# 처음 렌더링된 화면의 24개의 게시물을 크롤링하는 함수


def initial_crawling():
    urls = driver.find_elements_by_xpath(
        '//*[@id="react-root"]/section/main/article/div[2]/div/div/div/a')
    urls = [url.get_attribute("href") for url in urls]

    hash_tag_df = pd.DataFrame(columns=['time', 'hash_tag'])

    for i in range(len(urls)):
        req = requests.get(urls[i])
        response = TextResponse(req.url, body=req.text, encoding="utf-8")
        hash_tag_list = response.css("meta[property='instapp:hashtags']")
        hash_tag_list = ['#' + hash_tag.attrib['content']
                         for hash_tag in hash_tag_list]
        hash_tags = ', '.join(hash_tag_list)

        data = [
            {'time': 0, 'hash_tag': hash_tags}
        ]
        df = pd.DataFrame(data)
        hash_tag_df = hash_tag_df.append(df)

    return hash_tag_df.reset_index().drop(columns=['index'])

In [14]:
# 스크롤내리면서 12개씩 추가되는 게시물을 크롤링하는 함수 (두번째만 실행) - chrome 전체화면 기준
# 현재 브라우저 화면 크기에 따라 추가적으로 불러오는 div 갯수가 달라짐
# 전체화면 기준 div의 urls 총 갯수 = 24 -> 36 -> 45 -> 45 -> ...

def second_crawling():
    for i in range(0, 1):
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
        time.sleep(1.5)

        urls = driver.find_elements_by_xpath(
            '//*[@id="react-root"]/section/main/article/div[2]/div/div/div/a')
        urls = [url.get_attribute("href") for url in urls]

        hash_tag_df = pd.DataFrame(columns=['time', 'hash_tag'])

        for i in range(len(urls) - 12, len(urls)):
            req = requests.get(urls[i])
            response = TextResponse(req.url, body=req.text, encoding="utf-8")
            hash_tag_list = response.css("meta[property='instapp:hashtags']")
            hash_tag_list = ['#' + hash_tag.attrib['content']
                             for hash_tag in hash_tag_list]
            hash_tags = ', '.join(hash_tag_list)

            data = [
                {'time': 0, 'hash_tag': hash_tags}
            ]
            df = pd.DataFrame(data)
            hash_tag_df = hash_tag_df.append(df)

    return hash_tag_df.reset_index().drop(columns=['index'])

In [15]:
# 세번째부터는 이 함수 반복하기 - 게시물 9개씩 크롤링

def repeat_crawling():
    for i in range(0, 1):
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
        time.sleep(1.5)

        urls = driver.find_elements_by_xpath(
            '//*[@id="react-root"]/section/main/article/div[2]/div/div/div/a')
        urls = [url.get_attribute("href") for url in urls]

        hash_tag_df = pd.DataFrame(columns=['time', 'hash_tag'])

        for i in range(len(urls) - 9, len(urls)):
            req = requests.get(urls[i])
            response = TextResponse(req.url, body=req.text, encoding="utf-8")
            hash_tag_list = response.css("meta[property='instapp:hashtags']")
            hash_tag_list = ['#' + hash_tag.attrib['content']
                             for hash_tag in hash_tag_list]
            hash_tags = ', '.join(hash_tag_list)

            data = [
                {'time': 0, 'hash_tag': hash_tags}
            ]
            df = pd.DataFrame(data)
            hash_tag_df = hash_tag_df.append(df)

    return hash_tag_df.reset_index().drop(columns=['index'])

In [9]:
# 클래스로 모듈화

In [16]:
# %%writefile insta_crawling.py
# import requests
# import pandas as pd
# from selenium import webdriver
# from scrapy.http import TextResponse
# import seaborn as sns
# import getpass
# import time


class instagram_crawling():
    
    driver = ""
    
    def __init__(self):
        login_url = "https://www.instagram.com/accounts/login/?source=auth_switcher"
        self.driver = webdriver.Chrome()
        self.driver.get(login_url)
    
    
    # 로그인 함수 (seleniun webdriver)
    def login(self):

        my_id = input('id를 입력하세요: ')
        my_password = getpass.getpass('password를 입력하세요: ')

        self.driver.find_element_by_css_selector('._2hvTZ.pexuQ.zyHYP[type=text]').send_keys(my_id)
        self.driver.find_element_by_css_selector('._2hvTZ.pexuQ.zyHYP[type=password]').send_keys(my_password)
        self.driver.find_element_by_css_selector('.sqdOP.L3NKy.y3zKF[type=submit]').click()

        time.sleep(3)

        # 알림설정 하라는 modal창이 뜨는경우 '나중에 하기'를 클릭하는 코드
        alert_modal = self.driver.find_element_by_css_selector('body > div.RnEpo.Yx5HN > div > div')
        if alert_modal:
            self.driver.find_element_by_css_selector('body > div.RnEpo.Yx5HN > div > div > div.mt3GC > button.aOOlW.HoLwm').click()




    # 검색어를 입력하고 게시물이 가장 많은 검색결과를 클릭 (seleniun webdriver)
    def crawling_keyword(self, word):
        keyword = word

        self.driver.find_element_by_css_selector('#react-root > section > nav > div._8MQSO.Cx7Bp > div > div > div.LWmhU._0aCwM > input').send_keys(keyword)

        # 키워드 입력하고, 키워드에 대한 검색 리스트가 뜨는데 걸리는 시간만큼 기다려준다
        time.sleep(1.5)

        search_list = self.driver.find_elements_by_css_selector('#react-root > section > nav > div._8MQSO.Cx7Bp > div > div > div.LWmhU._0aCwM > div:nth-child(4) > div.drKGC > div > a')

        # 서칭된 a 엘리먼트들 중에 게시물 갯수가 가장 많은 a 엘리먼트를 뽑는다
        number_list = []

        for element in search_list:
            splited_element = element.text.split('게시물')
            if len(splited_element) == 2:
                number_list.append(int(re.sub(",", "", splited_element[1])))
            else:
                number_list.append(0)

        max_number = max(number_list)
        click_index = number_list.index(max_number)

        # 클릭하고 렌더링이 완료되도록 조금 기다리기
        search_list[click_index].click()
        time.sleep(1.5)




    # 처음 렌더링된 화면의 24개의 게시물을 크롤링하는 함수
    def initial_crawling(self):
        urls = self.driver.find_elements_by_xpath('//*[@id="react-root"]/section/main/article/div[2]/div/div/div/a')
        urls = [url.get_attribute("href") for url in urls]

        hash_tag_df = pd.DataFrame(columns=['time', 'hash_tag'])

        for i in range(len(urls)):
            req = requests.get(urls[i])
            response = TextResponse(req.url, body=req.text, encoding="utf-8")
            hash_tag_list = response.css("meta[property='instapp:hashtags']")
            hash_tag_list = ['#' + hash_tag.attrib['content'] for hash_tag in hash_tag_list]
            hash_tags = ', '.join(hash_tag_list)

            data = [
                {'time': 0, 'hash_tag': hash_tags}
            ]
            df = pd.DataFrame(data)
            hash_tag_df = hash_tag_df.append(df)

        return hash_tag_df.reset_index().drop(columns=['index'])




    # 스크롤내리면서 12개씩 추가되는 게시물을 크롤링하는 함수 (두번째만 실행) - chrome 전체화면 기준
    # 현재 브라우저 화면 크기에 따라 추가적으로 불러오는 div 갯수가 달라짐
    # 전체화면 기준 div의 urls 총 갯수 = 24 -> 36 -> 45 -> 45 -> ...
    def second_crawling(self):
        for i in range(0, 1):
            self.driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
            time.sleep(1.5)

            urls = self.driver.find_elements_by_xpath('//*[@id="react-root"]/section/main/article/div[2]/div/div/div/a')
            urls = [url.get_attribute("href") for url in urls]

            hash_tag_df = pd.DataFrame(columns=['time', 'hash_tag'])

            for i in range(len(urls) - 12, len(urls)):
                req = requests.get(urls[i])
                response = TextResponse(req.url, body=req.text, encoding="utf-8")
                hash_tag_list = response.css("meta[property='instapp:hashtags']")
                hash_tag_list = ['#' + hash_tag.attrib['content'] for hash_tag in hash_tag_list]
                hash_tags = ', '.join(hash_tag_list)

                data = [
                    {'time': 0, 'hash_tag': hash_tags}
                ]
                df = pd.DataFrame(data)
                hash_tag_df = hash_tag_df.append(df)

        return hash_tag_df.reset_index().drop(columns=['index'])



    # 세번째부터는 이 함수 반복하기 - 게시물 9개씩 크롤링
    def repeat_crawling(self):
        for i in range(0, 1):
            self.driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
            time.sleep(1.5)

            urls = self.driver.find_elements_by_xpath('//*[@id="react-root"]/section/main/article/div[2]/div/div/div/a')
            urls = [url.get_attribute("href") for url in urls]

            hash_tag_df = pd.DataFrame(columns=['time', 'hash_tag'])

            for i in range(len(urls) - 9, len(urls)):
                req = requests.get(urls[i])
                response = TextResponse(req.url, body=req.text, encoding="utf-8")
                hash_tag_list = response.css("meta[property='instapp:hashtags']")
                hash_tag_list = ['#' + hash_tag.attrib['content'] for hash_tag in hash_tag_list]
                hash_tags = ', '.join(hash_tag_list)

                data = [
                    {'time': 0, 'hash_tag': hash_tags}
                ]
                df = pd.DataFrame(data)
                hash_tag_df = hash_tag_df.append(df)

        return hash_tag_df.reset_index().drop(columns=['index'])
